In [ ]:
fold = 0

import pandas as pd

df = pd.read_csv(f"../../../training/discourse/chain-of-thought/wrong-predictions-fold-{fold}.csv")

df.head() # columns: text, label, id, test_fold

In [ ]:
all_entries = pd.read_csv("../../../input-data-all-together/discourse/sentence-only-with-folds.csv") 
all_entries = all_entries[all_entries["test_fold"] != fold]
# remove text-id column
all_entries = all_entries.drop(columns=["text-id"])
all_entries.head()

In [ ]:
# use sentence-transformers
from sentence_transformers import SentenceTransformer, util
import torch
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def rank_corpus_by_similarity(corpus, target_sentence):
    embeddings = model.encode(corpus, convert_to_tensor=True)
    target_embedding = model.encode(target_sentence, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(target_embedding, embeddings)[0]
    cos_scores = cos_scores.cpu()
    top_5_similar = torch.topk(cos_scores, k=5, largest=True)
    bottom_5_similar = torch.topk(cos_scores, k=5, largest=False)
    
    top_5_indices = top_5_similar.indices.numpy()
    bottom_5_indices = bottom_5_similar.indices.numpy()
    
    top_5_sentences = [corpus[i] for i in top_5_indices]
    bottom_5_sentences = [corpus[i] for i in bottom_5_indices]
    
    return top_5_sentences, bottom_5_sentences

In [ ]:
MAPPING = {
    0: "Evidence",
    1: "Claim",
    2: "Concluding Statement",
    3: "Lead",
    4: "Position",
    5: "Counterclaim",
    6: "Rebuttal"
}

developer_prompt = "You are an expert in analyzing persuasive essays and understanding argumentative and discourse elements. The discourse elements are: Lead, Position, Claim, Counterclaim, Rebuttal, Evidence, and Concluding Statement. Lead refers to an introduction that begins with a statistic, a quotation, a description, or some other device to grab the reader's attention and point toward the thesis. Position refers to an opinion or conclusion on the main question. Claim refers to a claim that supports the position. Counterclaim refers to a claim that refutes another claim or gives an opposing reason to the position. Rebuttal refers to a claim that refutes a counterclaim. Evidence refers to ideas or examples that support claims, counterclaims, rebuttals, or the position. Concluding statement refers to a concluding statement that restates the position and claims."
user_prompts = [] # {id: XX, prompt: YY}

# random seed
import random
random.seed(42)

id = 0
for index, row in df.iterrows():
    top_5_similar, bottom_5_similar = rank_corpus_by_similarity(all_entries[all_entries["label"] == row["label"]]["text"].tolist(), row["text"])
    sentences_to_use = [
        top_5_similar[0],
        top_5_similar[1],
        bottom_5_similar[-1],
        bottom_5_similar[-2],
        row["text"]
    ]

    for sentence_to_use in sentences_to_use:
        prompt = "This sentence is from the " + MAPPING[row["label"]] + " discourse element:\n" + sentence_to_use + "\n\nFirst, think step by step on why this is a sentence of the " + MAPPING[row["label"]] + " discourse element. Then, think step by step, and finally in the last line of your response (after putting a line break), please write a sentence that addresses the same topic, focusing on the " + MAPPING[row["label"]] + " discourse element. You should use different names, words, and terminologies in your output, but the overall meaning and content should be the same and refer to the same discourse element. In the *last line* of your output, just put the sentence and nothing else."
        user_prompts.append({"id": id, "prompt": prompt})
        id += 1


In [ ]:
len(user_prompts)

In [ ]:
print(user_prompts[1]["prompt"])

In [ ]:
import openai
from openai import OpenAI

with open("../../../api_key.txt", "r") as f:
    api_key = f.read().strip()

openai.api_key = api_key
openai_client = OpenAI(api_key=api_key)

def return_message_from_openai(messages, temperature = 1):
    global openai_client
    response = openai_client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

In [ ]:
import json

jsonl_data = []

for i, prompt in enumerate(user_prompts):
    jsonl_data.append({
        "custom_id": f"request-{i+1}-id-{prompt['id']}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-2024-08-06",
            "messages": [
                {"role": "developer", "content": developer_prompt},
                {"role": "user", "content": prompt["prompt"]}
            ],
            "temperature": 1.0
        }
    })

with open(f"cot-of-wrong-guesses-dissimilarity-of-cot-fold-{fold}.jsonl", "w") as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry) + "\n")


# Batch API

In [ ]:
batch_input_file = openai_client.files.create(
    file=open(f"cot-of-wrong-guesses-dissimilarity-of-cot-fold-{fold}.jsonl", "rb"),
    purpose="batch"
)

print(batch_input_file)

In [ ]:
batch_input_file_id = batch_input_file.id
created_batch = openai_client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": f"CoT of Wrong Guesses of CoT (fold {fold}) Discourse"
    }
)
created_batch.id, created_batch.status

In [ ]:
batch = openai_client.batches.retrieve(created_batch.id)
batch.status

In [ ]:
file_response = openai_client.files.content(batch.output_file_id)

In [ ]:
# make a new dataframe id,text-id,text,label,original_id,test_fold, by using the df we had in the beginning
result_df = pd.DataFrame(columns=["id", "text", "label", "original_id"])

lines = file_response.text.split("\n")
id = 0
for index, row in df.iterrows():
    for j in range(5):
        data = json.loads(lines[id])
        new_row = {
            "id": data["custom_id"],
            # "text-id": df[df["id"] == int(data["custom_id"].split("-")[-1])]["text-id"].values[0],
            "text": data["response"]["body"]["choices"][0]["message"]["content"],
            "label": row["label"],
            "original_id": id
            # "test_fold": df[df["id"] == int(data["custom_id"].split("-")[-1])]["test_fold"].values[0]
        }
        result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)
        id += 1


result_df.head()

In [ ]:
result_df.to_csv(f"cot-of-wrong-guesses-dissimilarity-of-cot-fold-{fold}.csv", index=False)